### Etapa 2 - Tokenizando os Dados

In [1]:
import nltk
from nltk.corpus import twitter_samples

In [2]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /home/josue/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [ ]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json') 

In [22]:
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(tweet_tokens[0][0])

#FollowFriday


### Etapa 3 - Normalizando os dados

In [ ]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

In [19]:
def lemmatize_sentence(token):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(token):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word,pos))
    return lemmatized_sentence

In [25]:
print(tweet_tokens[0])

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


In [24]:
print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


### Etapa 4 - Removendo o ruído dos dados

In [36]:
import string
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/josue/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
def remove_noise(tweets_tokens,stopwords = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweets_tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        tokens = lemmatizer.lemmatize(token,pos)
        if len(tokens)>0 and tokens not in string.punctuation and tokens.lower() not in stopwords:
            cleaned_tokens.append(tokens.lower())
    
    return cleaned_tokens


In [38]:
stop_words = stopwords.words('english') 
print(remove_noise(tweet_tokens[0], stop_words))


['#followfriday', '@france_inte', '@pkuchly57', '@milipol_paris', 'top', 'engage', 'member', 'community', 'week', ':)']


In [39]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))


In [40]:
print(positive_tweet_tokens[500])
print(positive_cleaned_tokens_list[500])


['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '@abzugame', '#fanart', ':d', 'https://t.co/bi8k8tb9ht']


### Etapa 5 - Determinando a densidade da palavra

In [41]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token 
all_pos_words = get_all_words(positive_cleaned_tokens_list)


In [42]:
from nltk import FreqDist 
freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))


[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


### Etapa 6 - Preparando dados para o modelo

#### Convertendo tokens em um dicionário

In [43]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens) 
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)


#### Divisão do conjunto de dados para treinamento e teste do modelo


In [44]:
import random 
positive_dataset = [(tweet_dict, "Positive")for tweet_dict in positive_tokens_for_model] 
negative_dataset = [(tweet_dict, "Negative")for tweet_dict in negative_tokens_for_model] 
dataset = positive_dataset + negative_dataset 
random.shuffle(dataset) 
train_data = dataset[:7000]
test_data = dataset[7000:]


### Etapa 7 - Criando e testando o modelo

In [45]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data) 
print("Accuracy is:", classify.accuracy(classifier, test_data)) 
print(classifier.show_most_informative_features(10))


Accuracy is: 0.9963333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2051.5 : 1.0
                      :) = True           Positi : Negati =   1654.7 : 1.0
                     sad = True           Negati : Positi =     25.1 : 1.0
                follower = True           Positi : Negati =     23.6 : 1.0
                  arrive = True           Positi : Negati =     19.6 : 1.0
                     x15 = True           Negati : Positi =     16.2 : 1.0
                followed = True           Negati : Positi =     15.3 : 1.0
               community = True           Positi : Negati =     14.5 : 1.0
                    glad = True           Positi : Negati =     11.9 : 1.0
                 welcome = True           Positi : Negati =     11.5 : 1.0
None


In [47]:
from nltk.tokenize import word_tokenize 
nltk.download('punkt')
custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."
custom_tokens = remove_noise(word_tokenize(custom_tweet)) 
print(classifier.classify(dict([token, True] for token in custom_tokens)))


[nltk_data] Downloading package punkt to /home/josue/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Negative


In [48]:
custom_tweet = 'Congrats #SportStar on your 7th best goal from last season winning goal of the year :) #Baller #Topbin #oneofmanyworldies'
custom_tokens = remove_noise(word_tokenize(custom_tweet)) 
print(classifier.classify(dict([token, True] for token in custom_tokens)))

Positive


In [49]:
custom_tweet = 'Thank you for sending my baggage to CityX and flying me to CityY at the same time. Brilliant service. #thanksGenericAirline'
custom_tokens = remove_noise(word_tokenize(custom_tweet)) 
print(classifier.classify(dict([token, True] for token in custom_tokens)))


Positive


In [52]:
import re, string 
def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\'(?:%[0-9a-fA-F][0-9a-fA-F]))+",'', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens


In [53]:
custom_tweet = 'I ordered just once from TerribleCo, they screwed up gain.'
custom_tokens = remove_noise(word_tokenize(custom_tweet)) 
print(classifier.classify(dict([token, True] for token in custom_tokens)))


Positive
